In [7]:
import time
import os
import numpy as np
import pandas as pd
import mpmath

In [8]:
from algorithms.alg1_timeline_simulation import Run_simulation

In [9]:
from experiment.DoE import *

# Generate design table

In [10]:
#Simple setup for testing purposes:
run_settings = {'F_priority_scheme':["FCFS","NPS"],#
            "days":[10],
            'F_number_of_agents':[3],
            'Repetition':list(range(0,10))}                #

# Generate a full factorial:
df=build_full_fact(run_settings)  

# Constants affecting the experiments:
df["Done"] = 0 #Flag if the final model has been trained
df["RUN"] = df.index + 1

# fix integer coding of string factor levels
variables = ["F_priority_scheme"]
df = fix_label_values(df, run_settings, variables)
df.drop("Name_fix",axis=1, inplace=True)
print(df)

# Save the settings to a file
#np.save("results/experiment_settings.npy", run_settings) 

#store the new experimental design
df.to_csv("results/experiments.csv", index=False)

   F_priority_scheme  days  F_number_of_agents  Repetition  Done  RUN
0               FCFS  10.0                 3.0         0.0     0    1
1                NPS  10.0                 3.0         0.0     0    2
2               FCFS  10.0                 3.0         1.0     0    3
3                NPS  10.0                 3.0         1.0     0    4
4               FCFS  10.0                 3.0         2.0     0    5
5                NPS  10.0                 3.0         2.0     0    6
6               FCFS  10.0                 3.0         3.0     0    7
7                NPS  10.0                 3.0         3.0     0    8
8               FCFS  10.0                 3.0         4.0     0    9
9                NPS  10.0                 3.0         4.0     0   10
10              FCFS  10.0                 3.0         5.0     0   11
11               NPS  10.0                 3.0         5.0     0   12
12              FCFS  10.0                 3.0         6.0     0   13
13               NPS

C:\Users\Mike\desktop\Portable_ABPS\experiment\DoE.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[variable].loc[run] = value


In [11]:
experiments = df
experiments.index = experiments.RUN.values
experiment_list = experiments.RUN.values

# Run some experiments

In [12]:
#Loop through all experiments in the initial list
for experiment_i in experiment_list:
    print("================================"*3)
    print("Starting experiment: ",experiment_i)
        
    # Load up the experiments for potential updates:
    experiments = pd.read_csv("results/experiments.csv")
    experiments.index = experiments.RUN.values
    
    # Get the number of the experiment
    RUN = experiments.RUN[experiment_i]
    
    # Bypass the experiment if it is already performed
    if experiments.Done[experiment_i] == 0:
        
        #create folder for the results
        os.mkdir("results/"+str(RUN))
        
        ####### Factors ######################
        # Convert the factors into the original level values
        
        F_priority_scheme = experiments[experiments.RUN == experiment_i]["F_priority_scheme"].values[0]
        F_number_of_agents = int(experiments.F_number_of_agents[RUN])
        days = int(experiments.days[RUN])
        
        
        """ run the simulation """
        start_time = time.time()
        evlog, arrived_cases, Case_DB = Run_simulation(agents=F_number_of_agents, 
                       P_scheme=F_priority_scheme, 
                       D=days,
                       seed=RUN)# to 01/01 2020
        
        evlog["RUN"] = RUN
        evlog.to_csv("results/"+str(RUN)+"/"+str(RUN)+"_log.csv",index=False)
        
        Case_DB["RUN"] = RUN        
        Case_DB.to_csv("results/"+str(RUN)+"/"+str(RUN)+"_case_DB.csv",index=False)
        
        end_time = time.time()
        Time_sec = end_time - start_time
        
        ####### Results ######################
        
        """
        avg_est_NPS
        avg_est_throughput_time
        avg_est_NPS_priority
        min_tracelen
        max_tracelen
        avg_initial_delay
        avg_activity_start_delay
        avg_duration_delayed
        
        """
        
        """ Calculate metrics on closed cases """
        
        evlog_closed = evlog.loc[evlog["case_status"]=="closed"]
        
        experiments.at[RUN, 'closed_avg_actual_NPS'] = np.mean(evlog_closed["actual_NPS"])
        experiments.at[RUN, 'closed_avg_actual_throughput_time'] = np.mean(evlog_closed["actual_throughput_time"])
        experiments.at[RUN, 'closed_avg_predicted_NPS'] = np.mean(evlog_closed["est_NPS"])
        experiments.at[RUN, 'closed_avg_predicted_throughput_time'] = np.mean(evlog_closed["est_throughput_time"])
        experiments.at[RUN, 'closed_avg_predicted_NPS_priority'] = np.mean(evlog_closed["est_NPS_priority"])
        
        experiments.at[RUN, 'closed_avg_initial_delay'] = np.mean(evlog_closed["initial_delay"])
        experiments.at[RUN, 'closed_avg_activity_start_delay'] = np.mean(evlog_closed["activity_start_delay"])
        experiments.at[RUN, 'closed_avg_duration_delayed'] = np.mean(evlog_closed["duration_delayed"])
        
        
        """ Calculate metrics on ALL cases """
        evlog_all = evlog
                
        experiments.at[RUN, 'all_avg_actual_NPS'] = np.mean(evlog_all["actual_NPS"])
        experiments.at[RUN, 'all_avg_actual_throughput_time'] = np.mean(evlog_all["actual_throughput_time"])
        experiments.at[RUN, 'all_avg_predicted_NPS'] = np.mean(evlog_all["est_NPS"])
        experiments.at[RUN, 'all_avg_predicted_throughput_time'] = np.mean(evlog_all["est_throughput_time"])
        experiments.at[RUN, 'all_avg_predicted_NPS_priority'] = np.mean(evlog_all["est_NPS_priority"])
        
        experiments.at[RUN, 'all_avg_initial_delay'] = np.mean(evlog_all["initial_delay"])
        experiments.at[RUN, 'all_avg_activity_start_delay'] = np.mean(evlog_all["activity_start_delay"])
        experiments.at[RUN, 'all_avg_duration_delayed'] = np.mean(evlog_all["duration_delayed"])
        
        """ other metrics """        
        experiments.at[RUN, 'cases_arrived'] = arrived_cases
        experiments.at[RUN, 'cases_closed'] = evlog.loc[evlog["case_status"]=="closed"]['case_id'].nunique()
        experiments.at[RUN, 'case_queued'] = Case_DB.loc[(Case_DB["case_queued"]== True)]['theta_idx'].nunique()                
        experiments.at[RUN, 'cases_assigned'] = evlog_all['case_id'].nunique()
        
        experiments.at[RUN, 'min_tracelen'] = np.min(evlog["event_no"])
        experiments.at[RUN, 'max_tracelen'] = np.max(evlog["event_no"])
        experiments.at[RUN, 'Simulation_duration_min'] = Time_sec/60
        
        #flag that experiment is done
        experiments.at[RUN, 'Done'] = 1
        experiments.to_csv("results/experiments.csv",index=False)
        
        ######################################
    
# Store results
experiments.to_csv("results/experiments.csv",index=False)

Starting experiment:  1
2018-07-01 00:00:00
day 0 start
day 1 start
day 2 start
day 3 start
day 4 start
day 5 start
day 6 start
day 7 start
day 8 start
day 9 start
generating eventlog...
Starting experiment:  2
2018-07-01 00:00:00
day 0 start
day 1 start
day 2 start
day 3 start
day 4 start
day 5 start
day 6 start
day 7 start
day 8 start
day 9 start
generating eventlog...
Starting experiment:  3
2018-07-01 00:00:00
day 0 start
day 1 start
day 2 start
day 3 start
day 4 start
day 5 start
day 6 start
day 7 start
day 8 start
day 9 start
generating eventlog...
Starting experiment:  4
2018-07-01 00:00:00
day 0 start
day 1 start
day 2 start
day 3 start
day 4 start
day 5 start
day 6 start
day 7 start
day 8 start
day 9 start
generating eventlog...
Starting experiment:  5
2018-07-01 00:00:00
day 0 start
day 1 start
day 2 start
day 3 start
day 4 start
day 5 start
day 6 start
day 7 start
day 8 start
day 9 start
generating eventlog...
Starting experiment:  6
2018-07-01 00:00:00
day 0 start
day 1 sta